## Imports and Dataset Setup

In [6]:
import pathlib, os, sys, operator, re, datetime
from functools import reduce
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from keras import layers
from keras import Model
from datasets import load_dataset
assert float(tf.__version__[:3]) >= 2.3

# Set random seeds for reproducibility
SEED = 3
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Set the desired activation function
AF = 'relu'

# Enable or disable GPU
# To fully disable it, we need to hide all GPU devices from Tensorflow
# Make sure GPU is disabled for this inference part of the lab
ENABLE_GPU = True
# tf.debugging.set_log_device_placement(True)

if not ENABLE_GPU:
    tf.config.set_visible_devices([], 'GPU')

# Print Python and TF version, and where we are running
print(f'Running on Python Version: {sys.version}')
print(f'Using Tensorflow Version: {tf. __version__}')
if not tf.config.experimental.list_physical_devices("GPU"):
    print('Running on CPU')
else:
    print(f'Using GPU at: {tf.test.gpu_device_name()} (of {len(tf.config.experimental.list_physical_devices("GPU"))} available)')


Running on Python Version: 3.6.8 (default, May 31 2023, 10:28:59) 
[GCC 8.5.0 20210514 (Red Hat 8.5.0-18)]
Using Tensorflow Version: 2.6.2
Using GPU at: /device:GPU:0 (of 1 available)


In [7]:

# If the dataset is gated/private, make sure you have run huggingface-cli login
# dataset = load_dataset("imagenet-1k", use_auth_token=True) # TODO ssz too much data



In [8]:
resnet18_model = tf.keras.applications.ResNet50()

resnet50_model = tf.keras.applications.ResNet50()




## Build Model

In [9]:
"""
Reference:
[1] K. He et al. Deep Residual Learning for Image Recognition. CVPR, 2016
[2] K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers:
Surpassing human-level performance on imagenet classification. In
ICCV, 2015.
"""

# Standard ResNet Block class
class ResBlock(Model):
    def __init__(self, channels, downsample=False):
        super().__init__()
        self.__channels = channels
        self.__downsample = downsample
        
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        # Use he_normal becuase activations are not linear for ReLU and PReLU and he_normal produces better results
        INIT_SCHEME = 'he_normal'

        if self.__downsample:
            # same kernel size, strides is 2, and shortcut is a Conv2D layer
            self.conv1 = layers.Conv2D(self.__channels, kernel_size=3, strides=2, padding='same', kernel_initializer=INIT_SCHEME)
            self.shortcut = keras.Sequential([
                layers.Conv2D(self.__channels, kernel_size=1, strides=2, padding='same', kernel_initializer=INIT_SCHEME),
                layers.BatchNormalization()
            ])
        else:
            # same kernel size, strides is 1, and shortcut is an identity
            self.conv1 = layers.Conv2D(self.__channels, kernel_size=3, strides=1, padding='same', kernel_initializer=INIT_SCHEME)
            self.shortcut = keras.Sequential()
            
        self.conv2 = layers.Conv2D(self.__channels, kernel_size=3, strides=1, padding='same', kernel_initializer=INIT_SCHEME)
    
    def call(self, input):
        shortcut = self.shortcut(input)
        
        io = self.conv1(input)
        io = layers.BatchNormalization()(io)
        io = layers.ReLU()(io)
        
        io = self.conv2(io)
        io = layers.BatchNormalization()(io)
        io = layers.ReLU()(io)
        
        io = io + shortcut
        return layers.ReLU()(io)


class ResNet18(Model):
    # num_classes is the number of classes in specific classification task
    def __init__(self, num_classes=1000):
        super().__init__()
        INIT_SCHEME = 'he_normal'
        
        self.layer0 = keras.Sequential([
            layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer=INIT_SCHEME),
            layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU()
        ], name='layer0')
        
        self.layer1 = keras.Sequential([
            ResBlock(64),
            ResBlock(64)
        ], name='layer1')
        
        self.layer2 = keras.Sequential([
            ResBlock(128, downsample=True),
            ResBlock(128)
        ], name='layer2')
        
        self.layer3 = keras.Sequential([
            ResBlock(256, downsample=True),
            ResBlock(256)
        ], name='layer3')
        
        self.layer4 = keras.Sequential([
            ResBlock(512, downsample=True),
            ResBlock(512)
        ], name='layer4')
        
        self.gap = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes, activation='softmax', kernel_initializer=INIT_SCHEME)
        
    def call(self, input):
        io = self.layer0(input)
        io = self.layer1(io)
        io = self.layer2(io)
        io = self.layer3(io)
        io = self.layer4(io)
        io = self.gap(io)
        io = self.fc(io)
        return io


resnet18_model = ResNet18()
resnet18_model.build(input_shape=(None, 224, 224, 3))
resnet18_model.summary()



Model: "res_net18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Sequential)          (None, 56, 56, 64)        9728      
_________________________________________________________________
layer1 (Sequential)          (None, 56, 56, 64)        147712    
_________________________________________________________________
layer2 (Sequential)          (None, 28, 28, 128)       525440    
_________________________________________________________________
layer3 (Sequential)          (None, 14, 14, 256)       2099456   
_________________________________________________________________
layer4 (Sequential)          (None, 7, 7, 512)         8393216   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  51